In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [2]:
random_state=42
cross_validation_folds = 5
n_jobs = 2
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
print('*'*40, "train")
print(train_df.info())
print('*'*40, 'test')
print(test_df.info())

**************************************** train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1

In [4]:
print('train', train_df.shape)
print('test', test_df.shape)

train (1460, 81)
test (1459, 80)


In [5]:
random_state=42
cross_validation_folds = 5
n_jobs = 2
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
test_ids = test_df.Id
y = train_df.SalePrice
train_df.drop(['SalePrice'], inplace=True, axis=1)
combine = [train_df, test_df]
combined = pd.concat([train_df, test_df])
def fill_column_with_median_value(df, column):
    """
    fill numerical columns with median value
    fill categorial columns with most common values
    """
    if df[column].dtype=='float64':
        median_value = combined[column].median()
    elif df[column].dtype=='object':
        median_value = combined[column].mode()
    df[column] = df[column].fillna(median_value)
def find_columns_with_na_values(df):
    for column in df.columns:
        if df.loc[df[column].isnull()].size>0:
            yield column
for dataset in combine:
    for column in find_columns_with_na_values(dataset):
        fill_column_with_median_value(dataset, column)
    dataset.drop(['Id'], inplace=True, axis=1)
X = pd.get_dummies(train_df)
X_test = pd.get_dummies(test_df)
missing_cols = set( X.columns ) - set( X_test.columns )
print("\tmissing columns", missing_cols)
print('drop missing columns')
# for column in missing_cols:
#     X.drop(column, inplace=True, axis=1)
print('X shape', X.shape)
print('X_test shape', X_test.shape)

	missing columns {'Exterior1st_Stone', 'Heating_Floor', 'GarageQual_Ex', 'RoofMatl_ClyTile', 'Exterior2nd_Other', 'RoofMatl_Metal', 'Exterior1st_ImStucc', 'Utilities_NoSeWa', 'PoolQC_Fa', 'RoofMatl_Roll', 'Condition2_RRAe', 'Condition2_RRNn', 'Electrical_Mix', 'HouseStyle_2.5Fin', 'Condition2_RRAn', 'Heating_OthW', 'MiscFeature_TenC', 'RoofMatl_Membran'}
drop missing columns
X shape (1460, 288)
X_test shape (1459, 270)


In [27]:
categorical_columns = list(filter(lambda col: train_df[col].dtype=='object', list(train_df.columns)))
mapping = dict()
# get the mapping between column labels to values
for column in categorical_columns:
    combined[column] = dataset[column].astype('category')
    mapping[column] = dict(zip(combined[column].cat.categories, range(len(combined[column].cat.categories))))
print(mapping)
for dataset in combine:
    dataset.replace(mapping, inplace=True)
print(dataset)

{'MSZoning': {'C (all)': 0, 'FV': 1, 'RH': 2, 'RL': 3, 'RM': 4}, 'Street': {'Grvl': 0, 'Pave': 1}, 'Alley': {'Grvl': 0, 'Pave': 1}, 'LotShape': {'IR1': 0, 'IR2': 1, 'IR3': 2, 'Reg': 3}, 'LandContour': {'Bnk': 0, 'HLS': 1, 'Low': 2, 'Lvl': 3}, 'Utilities': {'AllPub': 0}, 'LotConfig': {'Corner': 0, 'CulDSac': 1, 'FR2': 2, 'FR3': 3, 'Inside': 4}, 'LandSlope': {'Gtl': 0, 'Mod': 1, 'Sev': 2}, 'Neighborhood': {'Blmngtn': 0, 'Blueste': 1, 'BrDale': 2, 'BrkSide': 3, 'ClearCr': 4, 'CollgCr': 5, 'Crawfor': 6, 'Edwards': 7, 'Gilbert': 8, 'IDOTRR': 9, 'MeadowV': 10, 'Mitchel': 11, 'NAmes': 12, 'NPkVill': 13, 'NWAmes': 14, 'NoRidge': 15, 'NridgHt': 16, 'OldTown': 17, 'SWISU': 18, 'Sawyer': 19, 'SawyerW': 20, 'Somerst': 21, 'StoneBr': 22, 'Timber': 23, 'Veenker': 24}, 'Condition1': {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'PosA': 3, 'PosN': 4, 'RRAe': 5, 'RRAn': 6, 'RRNe': 7, 'RRNn': 8}, 'Condition2': {'Artery': 0, 'Feedr': 1, 'Norm': 2, 'PosA': 3, 'PosN': 4}, 'BldgType': {'1Fam': 0, '2fmCon': 1, 'Duple

In [34]:
train_df.PoolQC.values.unique()

AttributeError: 'numpy.ndarray' object has no attribute 'unique'

## Apply baseline model - Random Forest

In [11]:
? pd.fillna

Object `pd.fillna` not found.


In [45]:
rdf = RandomForestRegressor()
cv = cross_val_score(rdf, X, y, cv=cross_validation_folds, n_jobs=n_jobs, verbose=1)
print("baseline model with random forest, performance = {}".format(cv))
rdf.fit(X, y)

[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.8s finished


baseline model with random forest, performance = [0.86671635 0.79649265 0.86125422 0.88696039 0.80256034]


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [46]:
y_pred = rdf.predict(X_test)
submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": y_pred
})
submission.to_csv('house_price_result_baseline_random_forest.csv', index=False)

In [47]:
! kaggle competitions submit house-prices-advanced-regression-techniques -f house_price_result_baseline_random_forest.csv \
-m "baseline model with random forest, no feature engineering"

^C
User cancelled operation
